In [2]:
import os, sys

open_ai_key = '...'
os.environ['OPENAI_API_KEY'] = open_ai_key
sys.path = ['/Users/walder2/kg_uq/'] + sys.path
path_to_data = '/Users/walder2/kg_uq/recipe_data'

In [3]:
from kg_extraction import *

/Users/walder2/torch_geometric_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
websites = [
    "https://www.allrecipes.com/sweet-potato-dump-cake-recipe-8363654", 
    "https://www.allrecipes.com/chocolate-stout-cake-recipe-8426369",
    'https://www.allrecipes.com/sweet-potato-sheet-cake-recipe-8405784',
    'https://www.allrecipes.com/recipe/16607/cheesecake-brownies/',
    'https://www.allrecipes.com/recipe/211165/pumpkin-brownies/',
    'https://www.allrecipes.com/recipe/9566/fudge-brownies-i/',
    'https://www.allrecipes.com/recipe/9827/chocolate-chocolate-chip-cookies-i/',
    'https://www.allrecipes.com/recipe/26237/double-chocolate-chip-cookies/'
    
    
]
recipes = [
    'sweet potato dump cake',
    'chocolate stout cake',
    'sweet potato sheet cake',
    'cheesecake brownies',
    'pumpkin brownies',
    'fudge brownies',
    'chocolate chip cookies',
    'double chocolate chip cookies'
]

In [ ]:
scrape_recipe_websites(websites, recipes, data_dir = path_to_data)

In [ ]:
entity_types = {
    "recipe": 'https://schema.org/Recipe',
    "ingredient": "https://schema.org/recipeIngredient",
    "measurement": "https://schema.org/QuantitativeValue",  
}

relation_types = {
    "hasCharacteristic": "https://schema.org/additionalProperty",
    "hasColor": "https://schema.org/color",
    "hasMeasurement": "https://schema.org/hasMeasurement",
    "cookTime": "https://schema.org/cookTime",
    "recipeInstruction": "https://schema.org/recipeInstructions"
    
 }

In [ ]:
extract_ingredients_directions(data_dir = path_to_data)

In [ ]:
extract_recipe_kg(entity_types=entity_types, relation_types=relation_types, data_dir=path_to_data)

In [5]:
train, (kg, kg_idx), (node_map, emb_map) = kg_to_hetero(data_dir = path_to_data, embeddings_model=None, undirected=True)

In [6]:
kg

,head,head_type,relation,tail,tail_type
0,cheesecake brownies,recipe,hasIngredient,brownie mix,ingredient
1,cheesecake brownies,recipe,hasIngredient,water,ingredient
2,cheesecake brownies,recipe,hasIngredient,vegetable oil,ingredient
3,cheesecake brownies,recipe,hasIngredient,eggs,ingredient
4,cheesecake brownies,recipe,hasIngredient,cream cheese,ingredient
...,...,...,...,...,...
136,sweet potato sheet cake,recipe,recipeInstruction,add powdered sugar and beat until well combined,instruction
137,sweet potato sheet cake,recipe,recipeInstruction,spread the frosting evenly over the cooled cak...,instruction
138,sweet potato sheet cake,recipe,hasIngredient,sweet potatoes,ingredient
139,sweet potato sheet cake,recipe,hasIngredient,box mix,ingredient
